In [1]:
import pandas as pd
import numpy as np
from __future__ import division
import csv as csv

In [2]:
train = pd.read_csv('train.csv', header=0)

train['Pclass'].fillna(train['Pclass'].median(), inplace=True)

train['Fare'].fillna(train['Fare'].mean(), inplace=True)

train['Gender'] = train['Sex'].map( {'female':0, 'male':1} ).astype(int)
train['Gender'].fillna(train['Gender'].median(), inplace=True)

train['Agefill'] = train['Age']
for i in range(0,2):
    for j in range(0,3):
        median_age = train[(train['Gender'] == i) & (train['Pclass'] == (j+1))]['Age'].dropna().median()
        train.loc[ train.Age.isnull() & (train['Gender'] == i) & (train['Pclass'] == (j+1)) , 'Agefill'] = median_age 

train['FamilySize'] = train['SibSp'] + train['Parch']
train['FamilySize'].fillna(train['FamilySize'].mean(), inplace=True)

train['AgCl'] = train['Agefill'] * train['Pclass']
train['AgCl'].fillna(train['AgCl'].mean(), inplace=True)

train.dtypes[train.dtypes.map(lambda x: x=='object')]

train = train.drop(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis=1)
print train.info()
train = train.values

X = np.transpose(np.array([train[:,2], train[:,5], train[:,6], train[:,7],  train[:,8], train[:,9]]))
#X = np.transpose(np.array([train[:,2], train[:,6]]))
y = train[:,1]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
Gender         891 non-null int64
Agefill        891 non-null float64
FamilySize     891 non-null int64
AgCl           891 non-null float64
dtypes: float64(3), int64(7)
memory usage: 69.7 KB
None


In [3]:
X = X - np.mean(X, axis=0)
X = X / (np.max(X, axis=0) - np.min(X, axis=0))

In [4]:
from sklearn.cross_validation import train_test_split

X = np.repeat(X, 15000, axis=0)
y = np.repeat(y, 15000, axis=0)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

print X_train.shape
print X_val.shape
print y_train.shape
print y_val.shape

(10692000, 6)
(2673000, 6)
(10692000,)
(2673000,)


In [5]:
from sklearn import linear_model

model = linear_model.LogisticRegression()
model.fit(X_train, y_train)

print model.score(X_train, y_train)
print model.score(X_val, y_val)

0.80588795361
0.805628881407


In [6]:
from sklearn.metrics import precision_score, recall_score

pred_prob = model.predict_proba(X_val)[:,1]
pred_val = np.array((pred_prob > 0.5), dtype=int)

print precision_score(y_val, pred_val)
print recall_score(y_val, pred_val)

0.773391046103
0.6983485248


In [7]:
model.coef_

array([[-2.15009384,  1.42824407, -2.74881947, -2.91103777, -2.51938827,
        -0.80495369]])

In [8]:
test = pd.read_csv('test.csv', header=0)

test['Pclass'].fillna(test['Pclass'].median(), inplace=True)

test['Fare'].fillna(test['Fare'].mean(), inplace=True)

test['Gender'] = test['Sex'].map( {'female':0, 'male':1} ).astype(int)
test['Gender'].fillna(test['Gender'].median(), inplace=True)

test['Agefill'] = test['Age']
for i in range(0,2):
    for j in range(0,3):
        median_age = test[(test['Gender'] == i) & (test['Pclass'] == (j+1))]['Age'].dropna().median()
        test.loc[ test.Age.isnull() & (test['Gender'] == i) & (test['Pclass'] == (j+1)) , 'Agefill'] = median_age

test['FamilySize'] = test['SibSp'] + test['Parch']
test['FamilySize'].fillna(test['FamilySize'].mean(), inplace=True)

test['AgCl'] = test['Agefill'] * test['Pclass']
test['AgCl'].fillna(test['AgCl'].mean(), inplace=True)

test.dtypes[test.dtypes.map(lambda x: x=='object')]

test = test.drop(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis=1)
test = test.values

X_test = np.transpose(np.array([test[:,1], test[:,4], test[:,5], test[:,6],  test[:,7], test[:,8]]))
pred_test_prob = model.predict_proba(X_test)[:,1]
pred_test = np.array((pred_test_prob > 0.5), dtype=int)

In [9]:
prediction_file = open("schil_log_reg.csv", "wb")
prediction_file_object = csv.writer(prediction_file)
prediction_file_object.writerow(["PassengerId", "Survived"])

test_file = open('test.csv', 'rb')
test_file_object = csv.reader(test_file)
header = test_file_object.next()

i=0

for row in test_file_object:
    prediction_file_object.writerow([row[0], pred_test[i]])
    i = (i+1)
    
prediction_file.close()